In [1]:
import torch
import random
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Model Building
class CausalMaskAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.c_attn = nn.Linear(config.n_embd, 3*config.n_embd)
        self.register_buffer('bias', torch.tril(torch.ones(config.block_size, congig.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.c_attn(x)
        q, k, v = qkv.split(config.n_embd, dim = -1)
        q = q.view(B, T, config.n_heads, C//config.n_heads)